In [1]:
from googleapiclient.discovery import build
import csv
from datetime import datetime, timedelta
import networkx as nx
from itertools import combinations
import math

In [2]:
DEVELOPER_KEY = open("../esercizi_classe/api_key.txt").read()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

Ricerca e salvataggio di tutti i video per canale nel periodo temporale

In [3]:
channels = ["Romeo Agresti", "Il BiancoNero", "Colpo Gobbo","Luca Toselli","lAngolodiKinoshi"]

In [ ]:
def getIDfromName(name):
    request = youtube.search().list(
        part="snippet",
        q=name,
        type="channel",
        maxResults=5
    )
    response = request.execute()
    return response['items'][0]['id']['channelId']

def getChannelPlaylist(channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_videos_from_channel(playlist_id,channel_name):
    begin_date = datetime(2024,7,7)
    end_date = datetime(2025,3,24)
    video_ids_and_dates = []
    next_page_token = None
    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_date = datetime.strptime(item['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
            if begin_date <= video_date <= end_date:
                video_ids_and_dates.append((item['snippet']['resourceId']['videoId'], video_date))
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    with open(f"video_ids_{channel_name}.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        for video_id in video_ids_and_dates:
            writer.writerow([video_id[0],video_id[1].strftime("%Y-%m-%d")])
    print(f"Video IDs for {channel_name} saved to video_ids_{channel_name}.csv")

def get_csv_files(channels):
    for channel in channels:
        channel_id = getIDfromName(channel)
        playlist_id = getChannelPlaylist(channel_id)
        get_videos_from_channel(playlist_id,channel)
    

In [ ]:
get_csv_files(channels)

<p>Reperimento dei commenti:
<p>-per ogni commento salvo id, video commentato, autore, contenuto, data, likes, id commento a cui risponde (se c'è)

In [33]:
class Comment:
    def __init__(self, id, video_id, content, author,date, likes, reply_to_id=None):
        self.id = id
        self.video_id = video_id
        self.content = content
        self.author = author
        self.date = date
        self.likes = likes
        self.reply_to_id = reply_to_id

In [34]:
def get_comments_one_vid(video_id):
    comments = []
    request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId=video_id,
        textFormat="plainText",
        maxResults=100
    )
    response = request.execute()
    comments.extend(get_comments_from_response(response["items"]))
    next_page_token = response.get("nextPageToken", None)
    while next_page_token:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        response = request.execute()
        comments.extend(get_comments_from_response(response["items"]))
        next_page_token = response.get("nextPageToken", None)
    return comments

def get_comments_from_response(items):
    comments = []
    for item in items:
        main_comment = item["snippet"]["topLevelComment"]
        comment_id = main_comment["id"]
        video_id = item["snippet"]["videoId"]
        author = main_comment["snippet"]["authorDisplayName"]
        content = main_comment["snippet"]["textDisplay"]
        date = datetime.strptime(main_comment["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
        likes = main_comment["snippet"]["likeCount"]
        comments.append(Comment(comment_id, video_id, content, author, date, likes))
        if "replies" in item:
            for reply in item["replies"]["comments"]:
                reply_id = reply["id"]
                reply_content = reply["snippet"]["textDisplay"]
                reply_author = reply["snippet"]["authorDisplayName"]
                reply_date = datetime.strptime(reply["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
                reply_likes = reply["snippet"]["likeCount"]
                comments.append(Comment(reply_id, video_id, reply_content, reply_author, reply_date, reply_likes, comment_id))
    return comments

def save_comments_csv(comments, channel_name):
    with open(f"comments_{channel_name}.csv", "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Comment ID", "Video ID", "Content", "Author", "Date", "Likes", "Reply To ID"])
        for comment in comments:
            writer.writerow([comment.id, comment.video_id, comment.content, comment.author, comment.date.strftime("%Y-%m-%d"), comment.likes, comment.reply_to_id])
    print(f"Comments for {channel_name} saved to comments_{channel_name}.csv")

def get_comments_from_csv_file(channel):
    with open(f"video_ids_{channel}.csv", "r") as csvfile:
        reader = csv.reader(csvfile)
        video_ids = [row[0] for row in reader]
    all_comments = []
    for video_id in video_ids:
        comments = get_comments_one_vid(video_id)
        all_comments.extend(comments)
    save_comments_csv(all_comments, channel)

In [ ]:
for channel in channels:
    get_comments_from_csv_file(channel)

In [35]:
# Strutture ausiliarie:
# video_id -> canale
# video_id -> commentatori
# canale -> video_id
# canale -> commentatori
from collections import defaultdict

def video_id_to_channel(channels):
    video_id_to_channel = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                video_id_to_channel[row[0]] = channel
    return video_id_to_channel

def channel_to_video_id(channels):
    channel_to_video_id = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if channel not in channel_to_video_id:
                    channel_to_video_id[channel] = []
                channel_to_video_id[channel].append(row[0])
    return channel_to_video_id

def video_id_to_commenters(channels):
    video_id_to_commenters = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                video_id = row["Video ID"]
                author = row["Author"]
                if video_id not in video_id_to_commenters:
                    video_id_to_commenters[video_id] = set()
                video_id_to_commenters[video_id].add(author)
    return video_id_to_commenters

def channel_to_commenters(channels):
    channel_to_commenters = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if channel not in channel_to_commenters:
                    channel_to_commenters[channel] = set()
                channel_to_commenters[channel].add(row["Author"])
    return channel_to_commenters

def user_to_video_id(channels):
    user_to_video_id = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                author = row["Author"]
                video_id = row["Video ID"]
                if author not in user_to_video_id:
                    user_to_video_id[author] = set()
                user_to_video_id[author].add(video_id)
    return user_to_video_id

def channel_to_comments(channels):
    channel_to_comments = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                Comment(row["Comment ID"], row["Video ID"], row["Content"], row["Author"], row["Date"], row["Likes"], row["Reply To ID"])
                if channel not in channel_to_comments:
                    channel_to_comments[channel] = []
                channel_to_comments[channel].append(Comment(row["Comment ID"], row["Video ID"], row["Content"], row["Author"], row["Date"], row["Likes"], row["Reply To ID"]))
    return channel_to_comments

def video_to_date(channels):
    video_to_date = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                video_to_date[row[0]] = row[1]
    return video_to_date


Channel_of = video_id_to_channel(channels)
Videos_of = channel_to_video_id(channels)
Commenters_of_video = video_id_to_commenters(channels)
Commenters_of_channel = channel_to_commenters(channels)
Videos_commented_by = user_to_video_id(channels)
Comments_of_channel = channel_to_comments(channels)
Date_of_video = video_to_date(channels)

In [38]:
Comments_of_channel["Romeo Agresti"][0].likes

'0'

In [ ]:
class NodeCentralities:
    def __init__(self, node, degree, eigenvector_centrality, betweenness_centrality,degree_centrality):
        self.node = node
        self.degree = degree
        self.eigenvector_centrality = eigenvector_centrality
        self.betweenness_centrality = betweenness_centrality
        self.degree_centrality = degree_centrality

In [10]:
total_videos = []
for channel in channels:
    i = 0
    for video_id in Videos_of[channel]:
        if i == 50:
            break
        total_videos.append(video_id)
        i += 1        

50 video per canale. Cocommentatori Comuni

In [11]:
G = nx.Graph()

for video1, video2 in combinations(total_videos, 2):
    commenters1 = set(Commenters_of_video[video1])
    commenters2 = set(Commenters_of_video[video2])
    common_commenters = commenters1.intersection(commenters2)
    if len(common_commenters) > 1:
        G.add_edge(video1, video2, weight=len(common_commenters))
for n in G.nodes:
    G.nodes[n]['channel'] = Channel_of[n]
    G.nodes[n]['date'] = Date_of_video[n]

len(G.nodes), len(G.edges)
nx.write_gexf(G,"cocommentatori_50_video.gexf")

In [ ]:
media_pesi = 0
media_pesi_stesso_canale = 0
counter = 0

for edge in G.edges(data=True):
    sorgente = edge[0]
    destinazione = edge[1]
    if G.nodes[sorgente]['channel'] == G.nodes[destinazione]['channel']:
        media_pesi_stesso_canale += edge[2]['weight']
        counter += 1
        continue
    media_pesi += edge[2]['weight']

media_pesi_stesso_canale /= counter
media_pesi /= len(G.edges)
print("Media pesi:", media_pesi)
print("Media pesi stesso canale:", media_pesi_stesso_canale)

In [ ]:
# weight distribution
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

def plot_weight_distribution(G):
    weights = [data['weight'] for _, _, data in G.edges(data=True)]
    plt.figure(figsize=(10, 6))
    plt.plot(weights, 'o', markersize=2, alpha=0.5)
    plt.title('Weight Distribution of Edges')
    plt.xlabel('Weight')
    plt.ylabel('Frequency')
    # log scale
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
plot_weight_distribution(G)

In [ ]:
print(len(list(nx.local_bridges(G)))>0)

In [ ]:
import seaborn as sns
import scipy
degree=dict(nx.degree(G))
degree_centrality = {n: round(c/(G.order()-1),4) for n,c in degree.items()}
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality=nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G,200)
all_centr = [degree_centrality, closeness_centrality, betweenness_centrality, eigenvector_centrality]
nodes = list(G.nodes())

corrs = [
    [
        scipy.stats.pearsonr(
            [c1[n] for n in nodes],
            [c2[n] for n in nodes]
        )[0]
        for c1 in all_centr
    ]
    for c2 in all_centr
]

centralities = ["degree", "closeness", "betweenness", "eigenvector"]
len(betweenness_centrality)

In [15]:
total_videos = set(total_videos)
total_videos

{'-ynPYaA7GKU',
 '0SKFfmXRJg0',
 '0VKLfJ-ZFjQ',
 '0hY8er1UQdw',
 '0phKtYcp7EQ',
 '1N8hyowJGyI',
 '1RgHwkBjfPA',
 '1W1dJC3L1vE',
 '1ZBQDbFLN2Y',
 '2IQCGIbcUBI',
 '2Q7nQOkVwYI',
 '306OM1jBWPs',
 '3430yy32m34',
 '3EPEif7BmP4',
 '3kdCtGZG9iE',
 '3tR70K_6hdQ',
 '3y06thOOg1A',
 '4aIOWlym8GA',
 '4neAKH0whsY',
 '4wXMsJXr0s8',
 '50RhuQFos34',
 '519BtI_Hyng',
 '51eStzva8j0',
 '5DjqovdLuJo',
 '5KXECIcHj4Q',
 '5ljv7zy8Zes',
 '63w9vrAHB7g',
 '64vwWb--K5c',
 '6HaL5SA3KlI',
 '6mklcTlb3-k',
 '73qdMosRzxY',
 '7SKdV2kRc1k',
 '7dp10Ds96Ow',
 '8VSJaFJDqfI',
 '8XFjM56yhRA',
 '9-dqlIMbzag',
 '96Q_r1-oSZU',
 '9IzEuFESa2Q',
 '9UWESBA3SNA',
 'ALNO8KqU9yA',
 'AMEhy9rFgVs',
 'AZ-Coyzcm7I',
 'Ac28WB93csw',
 'Ae-NzOt_5VA',
 'AjuBHKrrJXg',
 'AuuteriYU-I',
 'Ax1rC1IgsyM',
 'Ay845KQdj1Y',
 'BVlH1XnZd90',
 'Bh5ieo2tNSo',
 'BqeUJ4KN8g0',
 'C-LPz1x3uAs',
 'C2sBtjoXbcg',
 'CFDwD7R07R4',
 'CY45aMoQVzA',
 'Cp2eRHcqs1Y',
 'DJgxZxfP3zQ',
 'DTGCZ9G8aiY',
 'DzdNpDtO9xU',
 'E2HNCwLOTFw',
 'E8X_h5iKZ_A',
 'ECoqE7VyBbI',
 'EI66cY

In [ ]:
# degree distribution
def plot_degree_distribution(G):
    degrees = [degree for node, degree in G.degree()]
    plt.figure(figsize=(10, 6))
    plt.hist(degrees, bins=30, color='blue', alpha=0.7)
    plt.title('Degree Distribution')
    plt.xlabel('Degree')
    plt.ylabel('Frequency')
    plt.show()
plot_degree_distribution(G)

In [20]:
Videos_commented_by["@RomeoAgresti"]

{'-GNlTMtX-EI',
 '-KQs4IYPmbA',
 '0nMK5d44Y4s',
 '0uMN1wNllbg',
 '0v5_3afPwYE',
 '1A2-tcCEPkA',
 '1b9q3Dh8W0c',
 '2onAxRQKX1Q',
 '3Ob6wFg3xXo',
 '4fpxYOvRsqw',
 '4pidcrXZl7U',
 '4xNWyrYeYoo',
 '5Ba9sFEEpZM',
 '5KXECIcHj4Q',
 '5R-BqXZxr3Y',
 '5U71G7D7qC4',
 '5ob4Fh_rqdw',
 '6AnJH82Xufc',
 '6mSjpZOB4RA',
 '72hR30RM1pI',
 '7PIWRqEgyk4',
 '86Xkj6Ub1hQ',
 '9Pv3XTMVLyc',
 'A8s10yLlatM',
 'AGF8KTQFBU0',
 'AzK9ZZqGQ2Y',
 'BHTS1VLnDyg',
 'BRNMyMYVNZs',
 'C2L5YQBatic',
 'C6viyyFAiik',
 'CI_AnYCW8G0',
 'Cw_PCkrU-LE',
 'DCHnD_gnAtw',
 'DbnZc36hdss',
 'DoOStdS8q6Y',
 'F6jV5tgrEhU',
 'GK5aFI7HeF4',
 'GPlFhUvBUsE',
 'HJfCRXr_U3k',
 'HfymSU0FvpU',
 'I-2dkBeod7c',
 'IG0ijE612h4',
 'IWmMO730dZ4',
 'IioYpLGcvW0',
 'JF8lGz6uljc',
 'JOFR_RcfU5g',
 'JRl2FnQNBK0',
 'JyYx2j7cVlE',
 'JzrygMdAEYo',
 'KSXBe7Q_opY',
 'KzHKFjED4_M',
 'LMHu31LDqPM',
 'Lcp122BfW5w',
 'LkkwQ1KcIME',
 'MNrR6KwGpuc',
 'MOF6lHuaZrA',
 'NLzByRitGQQ',
 'NZ9fy2btAbA',
 'NqH4j3V62Uk',
 'Nvg9bsfMizI',
 'OVQPdg7_J2o',
 'P905FSH4WFI',
 'PDaq0I

In [ ]:
plt.figure(figsize = (10,5))
sns.heatmap(pd.DataFrame(corrs),annot = True,cmap = 'coolwarm')
plt.xticks([x+0.5 for x in range(4)],centralities, rotation = 0)
plt.yticks([x+0.5 for x in range(4)],centralities, rotation = 0)
plt.title('Pearson correlation\n', weight = 'bold')
plt.show()

In [26]:
G_utenti = nx.Graph()
utenti = Videos_commented_by.keys()
for user1, user2 in combinations(utenti, 2):
    videos1 = Videos_commented_by[user1]
    videos2 = Videos_commented_by[user2]
    common_videos = videos1.intersection(videos2)
    common_videos = common_videos.intersection(total_videos)
    if len(common_videos) > 0:
        G_utenti.add_edge(user1, user2, weight=len(common_videos))
nx.write_gexf(G_utenti,"co-commentatori_utenti_piu_grande.gexf")

In [ ]:
len(G_utenti.nodes), len(G_utenti.edges) 
# (37916, 23021975) len > 0
# (21805, 5129502) len > 1
# (5541, 544245) len > 1 e intersezione con 50 video per canale
# (11873, 4373890) len > 0 e intersezione con 50 video per canale

(11873, 4373890)

In [39]:
Motta_comments = []
for channel in channels:
    with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if "motta" in row["Content"].lower() or "tiago" in row["Content"].lower() or "thiago" in row["Content"].lower() :
                Motta_comments.append(row)

In [ ]:
Motta_comments